In [162]:
# type: ignore

try:
  import sys
  sys.path.append('../')
  from modules.search_prop import *
except ImportError:
  url = 'https://raw.githubusercontent.com/jhjeong314/Proofmood/main/modules'
  import httpimport
  with httpimport.remote_repo(url):
    from search_prop import *

from pprint import pprint
import copy

In [163]:
# NodeLabel.build_str() test
# blank conclusion and blank hypothesis example

label1 = NodeLabel(LabelType.FORMULA, 'top .')
label2 = NodeLabel(LabelType.FORMULA, 'top')
print(label1 == label2)
print()
print(label1.build_str())
print()
print(label2.build_str())
label3 = NodeLabel(LabelType.FORMULA, 'top .hyp')
label4 = NodeLabel(LabelType.FORMULA, '.hyp')
print()
print(label3 == label4)
print()
print(label3.build_str())
print()
print(label4.build_str())

True

type: formula
line: top .
formula: top
ann: 
is_hyp: False

type: formula
line: top
formula: top
ann: 
is_hyp: False

True

type: formula
line: top .hyp
formula: top
ann: hyp 
is_hyp: True

type: formula
line: .hyp
formula: top
ann: hyp 
is_hyp: True


In [164]:
# Create a blank proof.
proof = ProofNodeS() # blank proof
proof.show_fitch_text()

│1. 	 .hyp
├─
│2.  


In [165]:
# Set hypothesis.
proof.update_formula(1, Formula('A'))
proof.show_fitch_text()

│1. A	 .hyp
├─
│2.  


In [166]:
# Set conclusion.
proof.update_formula(2, Formula('B imp A'))
proof.annotate(2, Ann('imp intro 1-2')) 
#^ this annotation is tentative and incorrect
proof.show_fitch_text()

│1. A	 .hyp
├─
│2. B imp A	x imp intro 1-2


In [167]:
# insert a subproof

# Please note that the premise in the annotation of line 2 is 
# automatically updated from "1-2" to "1-4" to reflect the 
# insertion of a subproof.

blank_proof = ProofNodeS() # blank proof to be inserted
proof.insert_node(2, blank_proof)
proof.show_fitch_text()

│1. A	 .hyp
├─
││2. 	 .hyp
│├─
││3.  
│4. B imp A	x imp intro 1-4


In [168]:
# Update the inserted subproof.
proof.update_formula('2', Formula('B'))
proof.update_formula('3', Formula('A'))
proof.annotate('3', Ann('repeat 1'))
# Annotate line 4 correctly.
proof.annotate('4', Ann('imp intro 2-3'))
proof.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. A	✓ repeat 1
│4. B imp A	✓ imp intro 2-3


In [169]:
# insert a blank line
prf = copy.deepcopy(proof)
prf.insert_node('3') 
# note the automatic adjustments of annotations
prf.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. 
││4. A	✓ repeat 1
│5. B imp A	✓ imp intro 2-4


In [170]:
# delete a line
prf2 = copy.deepcopy(prf)
prf2.delete_node(3)
# note the automatic adjustments of annotations
prf2.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. A	✓ repeat 1
│4. B imp A	✓ imp intro 2-3


In [171]:
# Replace the blank line (Line 3) of prf (not of pref2) 
# with a comment.
label = NodeLabel(type=LabelType.COMMENT_CONC, 
                  line='# blank line replaced')
new_node = ProofNode(label)
prf.replace_node(3, new_node)
prf.show_fitch_text()

│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. # blank line replaced
││4. A	✓ repeat 1
│5. B imp A	✓ imp intro 2-4


In [172]:
# ProofNode.build_str() test

prf.show_fitch_text()
print()

if prf.index_dict is not None:
  for key in prf.index_dict:
    p_node = prf.get_p_node(key)
    print(p_node.build_str())
    print()


│1. A	 .hyp
├─
││2. B	 .hyp
│├─
││3. # blank line replaced
││4. A	✓ repeat 1
│5. B imp A	✓ imp intro 2-4

label.type: subproof
label.is_hyp: False
line_num: 1-5
index: [0]
number of children: 3

label.type: formula
label.is_hyp: True
line_num: 1
index: [0, 0]
label.line: A	 .hyp 
validated: True

label.type: subproof
label.is_hyp: False
line_num: 2-4
index: [0, 1]
number of children: 3

label.type: formula
label.is_hyp: True
line_num: 2
index: [0, 1, 0]
label.line: B	 .hyp 
validated: True

label.type: comment.conc
label.is_hyp: False
line_num: 3
index: [0, 1, 1]
label.line: # blank line replaced
validated: True

label.type: formula
label.is_hyp: False
line_num: 4
index: [0, 1, 2]
label.line: A	 .repeat 1
validated: True

label.type: formula
label.is_hyp: False
line_num: 5
index: [0, 2]
label.line: B imp A	 .imp intro 2-4
validated: True



In [173]:
# Let's work on the following proof.
# (B and C imp A) proves (B imp A) or (C imp A)

prf_str = """
1. B and C imp A .hyp
proves
2. C or not C
  3. C .hyp
  proves
    4. B .hyp
    proves
    5. B and C 
    6. A 
  7. B imp A 
  8. (B imp A) or (C imp A) 
  9. not C .hyp
  proves
    10. C .hyp
    proves
    11. bot 
    12. A 
  13. C imp A 
  14. (B imp A) or (C imp A) 
15. (B imp A) or (C imp A) 
"""
proof = ProofNodeS(parse_fitch(prf_str))
proof.search_proof() # automatic annotation
proof.show_fitch_text()


All formulas have been validated.

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 9,10
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [174]:
# Build above proof from scratch.
prf = ProofNodeS()
# Set the hypothesis and the conclusion.
prf.update_formula(1, Formula('B and C imp A'))
prf.update_formula(2, Formula('(B imp A) or (C imp A)'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. (B imp A) or (C imp A)	x 


In [175]:
# As the conclusion is a disjunction formula, we will 
# explore the 'or elim' rule and the LEM rule, 
# in accordance with our heuristic.
# We need an LEM formula and two subproofs.
# First, we insert a blank proof.
subproof = ProofNodeS()
prf.insert_node(2, subproof)
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
││2. 	 .hyp
│├─
││3.  
│4. (B imp A) or (C imp A)	x 


In [176]:
# Then we add a line for the LEM formula.
# This is an interesting part. When we add a line in a subproof's 
# hypothesis part, it is put into the parent subproof's conclusion 
# part unless the line is a comment.
prf.insert_node(2)
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. 
││3. 	 .hyp
│├─
││4.  
│5. (B imp A) or (C imp A)	x 


In [177]:
# This is just a test which is not directly related 
# to our goal.  A subproof may have multiple lines
# when we insert comments.
label = NodeLabel(type=LabelType.COMMENT_HYP, 
                  line='# comment in a subproof is allowed')
new_node = ProofNode(label)
prf.insert_node(3, new_node)
prf.show_fitch_text()


│1. B and C imp A	 .hyp
├─
│2. 
││3. # comment in a subproof is allowed
││4. 	 .hyp
│├─
││5.  
│6. (B imp A) or (C imp A)	x 


In [178]:
# Delete the temporary comment line 3.
prf.delete_node(3)
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. 
││3. 	 .hyp
│├─
││4.  
│5. (B imp A) or (C imp A)	x 


In [179]:
# Let's proceed as planned.
# We set the LEM formula and insert another blank subproofs.
subproof = ProofNodeS()
prf.insert_node(5, subproof)
prf.update_formula(2, Formula('C or not C'))
prf.annotate(2, Ann('LEM'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. 	 .hyp
│├─
││4.  
││5. 	 .hyp
│├─
││6.  
│7. (B imp A) or (C imp A)	x 


In [180]:
# Fill in formulas in the subproofs.
prf.update_formula(3, Formula('C'))
prf.update_formula(4, Formula('(B imp A) or (C imp A)'))
prf.update_formula(5, Formula('not C'))
prf.update_formula(6, Formula('(B imp A) or (C imp A)'))
# Annotate line 7.
prf.annotate(7, Ann('or elim 2,3-4,5-6'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
││4. (B imp A) or (C imp A)	x 
││5. not C	 .hyp
│├─
││6. (B imp A) or (C imp A)	x 
│7. (B imp A) or (C imp A)	✓ or elim 2,3-4,5-6


In [181]:
# In the subproof 3-4, we try to obtain line 4 by 'or intro'
# applied to B imp A.
# In the subproof 5-6, we try to line 6 by 'or intro'
# applied to C imp A.
# Each disjunct is an implication formula. So we need 
# two subproofs for 'imp intro'.
subproof = ProofNodeS()
prf.insert_node(4, subproof)
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. 	 .hyp
││├─
│││5.  
││6. (B imp A) or (C imp A)	x 
││7. not C	 .hyp
│├─
││8. (B imp A) or (C imp A)	x 
│9. (B imp A) or (C imp A)	✓ or elim 2,3-6,7-8


In [182]:
# The function `insert_node()` does not handle the task of 
# duplicating the node (i.e., subproof). We must perform 
# the node duplication manually.
subproof = ProofNodeS() 
prf.insert_node(8, subproof) # note the line number 8
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. 	 .hyp
││├─
│││5.  
││6. (B imp A) or (C imp A)	x 
││7. not C	 .hyp
│├─
│││8. 	 .hyp
││├─
│││9.  
││10. (B imp A) or (C imp A)	x 
│11. (B imp A) or (C imp A)	✓ or elim 2,3-6,7-10


In [183]:
# Fill in the blank lines in the added subproofs.
prf.update_formula(4, Formula('B'))
prf.update_formula(5, Formula('A'))
prf.update_formula(9, Formula('C'))
prf.update_formula(9, Formula('A'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. A	x 
││6. (B imp A) or (C imp A)	x 
││7. not C	 .hyp
│├─
│││8. 	 .hyp
││├─
│││9. A	x 
││10. (B imp A) or (C imp A)	x 
│11. (B imp A) or (C imp A)	✓ or elim 2,3-6,7-10


In [184]:
# Work on subproof 3-6.
label1 = NodeLabel(type=LabelType.FORMULA, line='B and C .and intro 3,4')
print(label1.build_str())


type: formula
line: B and C .and intro 3,4
formula: B and C
ann: and intro 3,4
is_hyp: False


In [185]:
node1 = ProofNode(label1)
prf.insert_node(5, node1)
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	x 
││7. (B imp A) or (C imp A)	x 
││8. not C	 .hyp
│├─
│││9. 	 .hyp
││├─
│││10. A	x 
││11. (B imp A) or (C imp A)	x 
│12. (B imp A) or (C imp A)	✓ or elim 2,3-7,8-11


In [186]:
prf.annotate(6, Ann('imp elim 1,5'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. (B imp A) or (C imp A)	x 
││8. not C	 .hyp
│├─
│││9. 	 .hyp
││├─
│││10. A	x 
││11. (B imp A) or (C imp A)	x 
│12. (B imp A) or (C imp A)	✓ or elim 2,3-7,8-11


In [187]:
label2 = NodeLabel(type=LabelType.FORMULA, line='B imp A .imp intro 4-6')
prf.insert_node(7, ProofNode(label2))
prf.annotate(8, Ann('or intro 7'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. 	 .hyp
││├─
│││11. A	x 
││12. (B imp A) or (C imp A)	x 
│13. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-12


In [188]:
# Work on subproof 9-12.
label3 = NodeLabel(type=LabelType.FORMULA, line='bot')
prf.insert_node(11, ProofNode(label3))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. 	 .hyp
││├─
│││11. bot	x 
│││12. A	x 
││13. (B imp A) or (C imp A)	x 
│14. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-13


In [189]:
prf.update_formula(10, Formula('C'))
prf.annotate(11, Ann('bot intro 10,9'))
prf.annotate(12, Ann('bot elim 11'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. (B imp A) or (C imp A)	x 
│14. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-13


In [190]:
# Finish the proof.
label4 = NodeLabel(type=LabelType.FORMULA, line='C imp A .imp intro 10-12')
prf.insert_node(13, ProofNode(label4))
prf.annotate(14, Ann('or intro 13'))
prf.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [191]:
# cut-and-paste example 1
# cut_node() followed by insert_node()
prf99 = copy.deepcopy(prf)
node2 = prf99.cut_node('10-12')
prf99.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
││10. C imp A	x imp intro 
││11. (B imp A) or (C imp A)	✓ or intro 10
│12. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-11


In [192]:
prf99.insert_node(10, node2)
prf99.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	x imp intro 
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [193]:
prf99.annotate(13, Ann('imp intro 10-12'))
prf99.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [194]:
# cut-and-paste 2nd example
prf99 = copy.deepcopy(prf)
node3 = prf99.cut_node('3-8')
prf99.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. not C	 .hyp
│├─
│││4. C	 .hyp
││├─
│││5. bot	✓ bot intro 4,3
│││6. A	✓ bot elim 5
││7. C imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
│9. (B imp A) or (C imp A)	x or elim 2,3-8


In [195]:
# go_above = False case
prf99.insert_node(2, node3, go_above=False)
prf99.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	x or elim 8,9-14


In [196]:
prf99.annotate(15, Ann('or elim 2,3-8, 9-14'))
prf99.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [197]:
# cut-and-paste example 3
prf00 = copy.deepcopy(prf99)
p_node = prf00.cut_node(14)
prf00.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
│14. (B imp A) or (C imp A)	x or elim 2,3-8


In [198]:
prf00.insert_node(13, p_node, go_above=False)
prf00.annotate(15, Ann('or elim 2,3-8, 9-14'))
prf00.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [199]:
# move_node() test
prf00.move_node(14, 2)
prf00.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. (B imp A) or (C imp A)	x or intro 13
│3. C or not C	✓ LEM 
││4. C	 .hyp
│├─
│││5. B	 .hyp
││├─
│││6. B and C	✓ and intro 4,5
│││7. A	✓ imp elim 1,6
││8. B imp A	✓ imp intro 5-7
││9. (B imp A) or (C imp A)	✓ or intro 8
││10. not C	 .hyp
│├─
│││11. C	 .hyp
││├─
│││12. bot	✓ bot intro 11,10
│││13. A	✓ bot elim 12
││14. C imp A	✓ imp intro 11-13
│15. (B imp A) or (C imp A)	x or elim 3,4-9


In [200]:
# move back
prf00.move_node(2, 14, go_above=False)
prf00.annotate(15, Ann('or elim 2,3-8, 9-14'))
prf00.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
│││12. A	✓ bot elim 11
││13. C imp A	✓ imp intro 10-12
││14. (B imp A) or (C imp A)	✓ or intro 13
│15. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-14


In [201]:
# duplicate_node() test

prf00.duplicate_node(14, 2)
prf00.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. (B imp A) or (C imp A)	x or intro 13
│3. C or not C	✓ LEM 
││4. C	 .hyp
│├─
│││5. B	 .hyp
││├─
│││6. B and C	✓ and intro 4,5
│││7. A	✓ imp elim 1,6
││8. B imp A	✓ imp intro 5-7
││9. (B imp A) or (C imp A)	✓ or intro 8
││10. not C	 .hyp
│├─
│││11. C	 .hyp
││├─
│││12. bot	✓ bot intro 11,10
│││13. A	✓ bot elim 12
││14. C imp A	✓ imp intro 11-13
││15. (B imp A) or (C imp A)	✓ or intro 14
│16. (B imp A) or (C imp A)	✓ or elim 3,4-9,10-15


In [202]:
prf00 = copy.deepcopy(prf)
prf00.duplicate_node('4-6', 12)
prf00.show_fitch_text()

│1. B and C imp A	 .hyp
├─
│2. C or not C	✓ LEM 
││3. C	 .hyp
│├─
│││4. B	 .hyp
││├─
│││5. B and C	✓ and intro 3,4
│││6. A	✓ imp elim 1,5
││7. B imp A	✓ imp intro 4-6
││8. (B imp A) or (C imp A)	✓ or intro 7
││9. not C	 .hyp
│├─
│││10. C	 .hyp
││├─
│││11. bot	✓ bot intro 10,9
││││12. B	 .hyp
│││├─
││││13. B and C	x and intro 3,4
││││14. A	x imp elim 1,5
│││15. A	✓ bot elim 11
││16. C imp A	✓ imp intro 10-15
││17. (B imp A) or (C imp A)	✓ or intro 16
│18. (B imp A) or (C imp A)	✓ or elim 2,3-8,9-17


Up to this point, we have performed editing operations on individual nodes. Now, we aim to work on *chunks*, which refer to a sequence of consecutive siblings sharing a common parent.